# Intermezzo: Sci-kit learn

## Uczenie za pomocą gotowych pakietów

Będziemy dziś korzystać z zaawansowanego pakietu do sieci neuronowych "Keras". 

### Instalacja
* Pod Windowsem powinna być zainstalowana wersja 0.16.
* Pod Linuxem powinna być zainstalowana wersja 0.15.
* Najnowsza oficjalna wersja to 0.17 (0.18 na githubie w wersji deweloperskiej)

* Pod Linuxem można zainstalować lokalnie:
        pip install --user sklearn
* Lub na własnym komputerze:
        sudo pip install sklearn
* Możliwe, że konieczne jest ponowne uruchomienie IPython, jeśli był uruchomiony podczas instalacji.

### Dokumentacja
* Pełna i obszerna dokumentacja na: http://scikit-learn.org/0.15 (wystarczy zmienić numer wersji zgodnie z zainstalowaną wersją, aby przejść do dokumentów dla odpowiedniej wersji)

### Zadania teoretyczne
* Zapoznaj się z pakietem Sci-Kit learn na podstawie dokumentacji. 
* Przeglądając spis API (http://scikit-learn.org/0.15/modules/classes.html), podaj listę dostępnych rodzajów klasyfikatorów i regresorów. Które metody poznaliśmy na wykładach? Które są nowe?
* Jakie inne tematy widzisz (niż klasyifikacja/regresja), które były omawiane na wykładach?

### Zadanie praktyczne
* Opracuj tutorial http://scikit-learn.org/0.15/tutorial/text_analytics/working_with_text_data.html w IPython Notebook (do wykonania w tym tygodniu do najbliższych ćwiczeń) 

Powyższe zadania są punktowane razem na 40 punktów.

### Zadania na te i następne ćwiczenia
* Opracuj zadanie Exercise 1 lub Exercise 2 (do wyboru) na za dwa tygodnie w postaci IPython Notebook. 
* Będzie można też pracować przy tym na najbliszych ćwiczeniach.
* Wymagania:
    * Dane do tych zadań trzeba zdobyć samodzielnie. Dopuszczam dzielenie się danymi, jeśli ktoś już coś opracował/znalazł.
    * Wybierz i sprawdź co najmiej cztery różne rodzaje klasyfikatorów/regresorów, z tego co najmniej dwa, który nie były dotąd omawiane na ćwiczeniach (np. SVM, Naiwny Bayes) lub wykładach (drzewa decyzyjne).
    * Zawsze wykonaj walidacje krzyżową (5-krotną) i przedstaw wyniki. 
    * Dodatkowe: Spróbuj wykonać optymalizację hiperparamtrów za pomocą GridSearch lub RandomSearch. Pomogło?

To zadanie będzie warte 40 punktów z następnych ćwiczeń + 20 punktów za szczególnie ładne opracowania.


## Rozwiązania
### Zadania teoretyczne
API *Sci-Kit learn* udostępnia zestaw regresorów, klasyfikatorów, algorytmów klastrowania, funkcji narzędziowych i zbiorów dancyh. W zestawie znajduje się także część metod przedstawianych na wykładzie z UMZ. Między innymi są to:
- Naiwny klasyfikator Bayesa
- Regresor liniowy
- Regresor logistyczny (wersja jedno i wielo wymiarowa)
- Klastrowanie metodą k-średnich



Oprócz powyższych, pakiet zawiera jeszcze dużo modułów do klastrowania danych, oraz do obliczeń statystycznych

### Zadania praktyczne - opracowanie tutorialu

In [3]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [11]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
# twenty_train to dataset typu "bunch" - składowe mogą być utrzymywane jak ze słownika lub przez notację obiektową

# Przykłąd - pokazuję kategorie artykułów/
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [13]:
# rozmiar danych
print len(twenty_train.data)
print len(twenty_train.filenames)

2257
2257


In [20]:
# 5 pierwszych linii pierwszego artukułu na liście
print("\n".join(twenty_train.data[0].split("\n")[:5]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14


In [21]:
# kategoria pierwszego artukułu na liście
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [24]:
#Worki słów
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [27]:
# Obiekty klasy CountVectorizer umożliwiają tworzenie N-gramów dla wybranych słów
count_vect.vocabulary_.get(u'belief') # licznik wystąpień

6426

In [29]:
# Znacznie lepszą miarą od liczby wystąpień jest częstość - miara uwzględniająca liczbę słów w dokumencie
# policzymy częstość za pomocą obiektu TfidTransformer
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [32]:
# liczba wystąpień słów oraz częstość słów mogą być cechami
# poniżej o tym jak zbudować prosty model klasyfikacji na podstawie takich cech
# Będziemy korzystać z klasyfikatora Bayesowskiego
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, twenty_train.target)

In [33]:
docs_new = ['God is almighty', 'OpenGL on the GPU is fast', 'Breast cancer']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is almighty' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
'Breast cancer' => sci.med


In [40]:
#Pipeline
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),])

# Trenowanie modelu jedną komendą - to co wcześniej
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [46]:
# Ocena dopasowania modelu
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
print "Accuracy fo Bayes classification ", np.mean(predicted == twenty_test.target)   

Accuracy fo Bayes classification  0.912782956059


In [45]:
# SUpport Vector Machines
#Pipeline dla klasyfikatora opartego na SVM
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5)),])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
print "Accuracy for SVM classification ", np.mean(predicted == twenty_test.target)  

Accuracy for SVM classification  0.912782956059


In [48]:
# Bardziej zaawansowana analiza dopasowania - obiekt typu metrics względniający inne charakterystyki niż średnia
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))


                        precision    recall  f1-score   support

           alt.atheism       0.96      0.80      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.89      0.96      0.92       398

           avg / total       0.92      0.91      0.91      1502



array([[255,  11,  15,  38],
       [  2, 379,   4,   4],
       [  3,  33, 356,   4],
       [  5,  10,   2, 381]])